<a href="https://colab.research.google.com/github/OmarMohammed88/heart_disease/blob/main/models/heart_disease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prepare Data

In [ ]:
pip install --upgrade gdown

In [ ]:
!gdown --id 1NF9P2nsA_5eX7_p_qSygZf864M0QN-ER

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1NF9P2nsA_5eX7_p_qSygZf864M0QN-ER
To: /content/I2b2_HeartDisease DataSet.rar
100% 31.4M/31.4M [00:01<00:00, 28.5MB/s]


In [ ]:
!unrar e '/content/I2b2_HeartDisease DataSet.rar' -d dataset/


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/I2b2_HeartDisease DataSet.rar

Creating    dataset                                                   OK
Extracting  dataset/i2b2_2014_schedule_revised.pdf                         0%  OK 
Extracting  dataset/i2b2_2014_workshop_papers.zip                          1%  2%  3%  4%  5%  6%  7%  8%  9% 10% 11% 12% 13% 14% 15% 16% 17% 18%  OK 
Extracting  dataset/i2b2_evaluation_scripts-master.zip                    18%  OK 
Extracting  dataset/Readme_For_Dataset.txt                                18%  OK 
Extracting  dataset/testing-RiskFactors-Complete-MAE.tar.gz               19% 20% 21% 22%  OK 
Extracting  dataset/testing-RiskFactors-Complete.tar.gz                   23% 24% 25% 26%  OK 
Extracting  dataset/testing-RiskFactors-Gold.tar.gz               

In [ ]:
!tar -xvf '/content/dataset/training-RiskFactors-Complete-Set1.tar.gz' 
!tar -xvf '/content/dataset/training-RiskFactors-Complete-Set2.tar.gz'
!tar -xvf '/content/dataset/testing-RiskFactors-Complete.tar.gz'

training-RiskFactors-Complete-Set1/
training-RiskFactors-Complete-Set1/369-03.xml
training-RiskFactors-Complete-Set1/299-01.xml
training-RiskFactors-Complete-Set1/302-02.xml
training-RiskFactors-Complete-Set1/327-04.xml
training-RiskFactors-Complete-Set1/297-04.xml
training-RiskFactors-Complete-Set1/301-02.xml
training-RiskFactors-Complete-Set1/225-02.xml
training-RiskFactors-Complete-Set1/398-04.xml
training-RiskFactors-Complete-Set1/283-04.xml
training-RiskFactors-Complete-Set1/256-02.xml
training-RiskFactors-Complete-Set1/330-02.xml
training-RiskFactors-Complete-Set1/256-01.xml
training-RiskFactors-Complete-Set1/240-03.xml
training-RiskFactors-Complete-Set1/302-04.xml
training-RiskFactors-Complete-Set1/337-05.xml
training-RiskFactors-Complete-Set1/358-01.xml
training-RiskFactors-Complete-Set1/396-04.xml
training-RiskFactors-Complete-Set1/321-01.xml
training-RiskFactors-Complete-Set1/291-02.xml
training-RiskFactors-Complete-Set1/274-02.xml
training-RiskFactors-Complete-Set1/322-02.xm

In [ ]:
import xml.etree.cElementTree as ET
import os
import nltk
import string
import random
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from pandas import DataFrame
import nltk.data
from nltk import sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
codefolder = os.path.dirname(os.path.abspath('__file__'))

datafolders = ['/training-RiskFactors-Complete-Set1/','/training-RiskFactors-Complete-Set2/']
filenames = []

for folder in datafolders:
    for file in os.listdir(str(codefolder)+folder):
        filename=os.fsdecode(os.fsencode((str(codefolder)+folder+file)))
        if filename.endswith(('.xml')):
            filenames.append(filename)
            
filenames[:5]

['/content/training-RiskFactors-Complete-Set1/338-04.xml',
 '/content/training-RiskFactors-Complete-Set1/297-02.xml',
 '/content/training-RiskFactors-Complete-Set1/227-01.xml',
 '/content/training-RiskFactors-Complete-Set1/274-01.xml',
 '/content/training-RiskFactors-Complete-Set1/326-04.xml']

In [ ]:
# use a 90/10 split
split_index = int(len(filenames)*0.9)
random.seed(42)
random.shuffle(filenames)

train_filenames = filenames[:split_index]
dev_filenames = filenames[split_index:]

In [ ]:
#test files
datafolder = '/testing-RiskFactors-Complete/'
test_filenames = []

for file in os.listdir(str(codefolder)+datafolder):
    filename=os.fsdecode(os.fsencode((str(codefolder)+datafolder+file)))
    if filename.endswith(('.xml')):
        test_filenames.append(filename)

test_filenames[:5]

['/content/testing-RiskFactors-Complete/199-04.xml',
 '/content/testing-RiskFactors-Complete/168-04.xml',
 '/content/testing-RiskFactors-Complete/235-05.xml',
 '/content/testing-RiskFactors-Complete/208-02.xml',
 '/content/testing-RiskFactors-Complete/114-02.xml']

## Define function to get sentences & labels

In [ ]:
def get_sentences(file):

    tree = ET.ElementTree(file=file)
    root = tree.getroot()

    text = root.find('TEXT').text
    sentences = [sent.split('\n') for sent in sent_tokenize(text) if sent!='\n']
    all_sentences = []

    for item in sentences:
        for sub_item in item:
            if sub_item.replace(' ','') != '':
                all_sentences.append(sub_item)    
    print("all_sentences:",all_sentences[:5])
    return all_sentences

In [ ]:
def process_file(file):
    
    # get all sentences in the file
    tree = ET.ElementTree(file=file)
    root = tree.getroot()

    text = root.find('TEXT').text
    sentences = [sent.split('\n') for sent in sent_tokenize(text) if sent!='\n']
    all_sentences = []

    for item in sentences:
        for sub_item in item:
            if sub_item.replace(' ','') != '':
                all_sentences.append(sub_item)    
                
    #all_sent = get_sentences(file)
    sent_label = {}

    sub_tags = []
    for item in root.find("TAGS"):
        if item.tag == 'PHI':
            pass
        elif item.tag == 'SMOKER':
            label = (item.tag + "." + item.attrib['status']).lower().replace(" ", "_")
        elif item.tag == 'FAMILY_HIST':
            label = (item.tag + "." + item.attrib['indicator']).lower().replace(" ", "_")
        elif item.tag == 'MEDICATION':
            label = (item.tag + "." + item.attrib['type1'] + "." + item.attrib['type2'] + "." + item.attrib['time']).lower().replace(" ", "_")
        else:
            label = (item.tag + "." + item.attrib['indicator'] + "." + item.attrib['time']).lower().replace(" ", "_")
        for sub_item in item.findall(item.tag):
            print("sub_item:", sub_item.tag)
            if ('text' in sub_item.attrib.keys()):
                print(sub_item.attrib['text'])
                sub_tags.append((sub_item.attrib['text'], label))


    count=0
    for sent in all_sentences:
        label='Other'
        for tag in set(sub_tags):                                                       
            if tag[0] in sent:
                label = tag[1]
                count += 1

        sent_label[sent] = label
        
    # return empty dict if no tag found in file
    # else, return the sentences with the labels
    if count==0:
        return {}
    else:
        return sent_label

In [ ]:
def get_TrainingData(filenames):
    train_files = []
    train_sent = []
    train_labels = []

    for file in filenames:
        file_data = process_file(file=file)
        for i in range(0, len(file_data)):
            train_files.append(file)
            
        for key, value in file_data.items():
            train_sent.append(key)
            train_labels.append(value)
    
    train_df = pd.DataFrame({'filename': train_files, 'sentence': train_sent, 'label': train_labels})
    return train_df

In [ ]:
df_train = get_TrainingData(train_filenames)
df_dev = get_TrainingData(dev_filenames)
df_test = get_TrainingData(test_filenames)

Streaming output truncated to the last 5000 lines.
sub_item: MEDICATION
ASA
sub_item: MEDICATION
ASA
sub_item: MEDICATION
Regular insulinSS 
sub_item: MEDICATION
Regular insulinS
sub_item: MEDICATION
Regular insulin
sub_item: MEDICATION
Plavix
sub_item: MEDICATION
Plavix
sub_item: MEDICATION
Regular insulinSS 
sub_item: MEDICATION
Regular insulinS
sub_item: MEDICATION
Regular insulin
sub_item: MEDICATION
Metoprolol
sub_item: MEDICATION
Metoprolol
sub_item: MEDICATION
Metoprolol
sub_item: MEDICATION
Metoprolol
sub_item: MEDICATION
Metoprolol
sub_item: MEDICATION
Metoprolol
sub_item: HYPERTENSION
Hypertension
sub_item: HYPERTENSION
Hypertension
sub_item: HYPERTENSION
Hypertension
sub_item: HYPERTENSION
Hypertension
sub_item: HYPERTENSION
Hypertension
sub_item: HYPERTENSION
Hypertension
sub_item: HYPERTENSION
Hypertension
sub_item: HYPERTENSION
Hypertension
sub_item: HYPERTENSION
Hypertension
sub_item: CAD
Cath  i) Occluded RCA, subtotal circumflex and 95% LAD and 90% stenosis of first di

In [ ]:
df_train_flair = df_train[['label', 'sentence']].copy()
df_train_flair['prefix'] = '__label__'
df_train_flair = df_train_flair[['prefix', 'label', 'sentence']]
df_train_flair['formatted'] = df_train_flair['prefix']+df_train_flair['label']+' '+df_train_flair['sentence']
df_train_flair['formatted'].head()

0               __label__Other Record date: 2095-03-19
1                            __label__Other Admit Note
2                  __label__Other Dr. Vernell Fournier
3    __label__Other Ms. Francis is a Spanish speaki...
4    __label__Other Hx obtaiend from prior LMR note...
Name: formatted, dtype: object

In [ ]:
df_dev_flair = df_dev[['label', 'sentence']].copy()
df_dev_flair['prefix'] = '__label__'
df_dev_flair = df_dev_flair[['prefix', 'label', 'sentence']]
df_dev_flair['formatted'] = df_dev_flair['prefix']+df_dev_flair['label']+' '+df_dev_flair['sentence']
df_dev_flair['formatted'].head()

0               __label__Other Record date: 2065-04-28
1    __label__Other                      MEDIQUIK E...
2    __label__Other RUTLEDGE,XAVIER W.   539-54-14-...
3    __label__smoker.unknown This patient was seen ...
4                                  __label__Other The 
Name: formatted, dtype: object

In [ ]:
df_test_flair = df_test[['label', 'sentence']].copy()
df_test_flair['prefix'] = '__label__'
df_test_flair = df_test_flair[['prefix', 'label', 'sentence']]
df_test_flair['formatted'] = df_test_flair['prefix']+df_test_flair['label']+' '+df_test_flair['sentence']
df_test_flair['formatted'].head()

0        __label__Other Record date: 2093-01-10
1    __label__Other   CARDIOLOGY ADMISSION NOTE
2                     __label__Other Uriah LARA
3                 __label__Other FH 621-15-64-4
4     __label__Other Date of admission 01/10/93
Name: formatted, dtype: object

In [ ]:
df_train_flair['formatted'].to_csv('train.txt', index=False, header=False)
df_dev_flair['formatted'].to_csv('dev.txt', index=False, header=False)
df_test_flair['formatted'].to_csv('test.txt', index=False, header=False)


In [ ]:
le = LabelEncoder()

# refer (https://blog.insightdatascience.com/using-bert-for-state-of-the-art-pre-training-for-natural-language-processing-1d87142c29e7)

# get the training set for BERT in the required format
df_train_bert = pd.DataFrame({'user_id':df_train.index,
            'label':le.fit_transform(df_train['label']),
            'alpha':['a']*df_train.shape[0],
            'text':df_train['sentence'].replace(r'\n',' ',regex=True)})


# get the dev set for BERT in the required format
df_dev_bert = pd.DataFrame({'user_id':df_dev.index,
            'label':le.fit_transform(df_dev['label']),
            'alpha':['a']*df_dev.shape[0],
            'text':df_dev['sentence'].replace(r'\n',' ',regex=True)})


 # Creating test dataframe according to BERT (CoLA Format)
df_test_bert = pd.DataFrame({'id':df_test.index,
                              'label':le.fit_transform(df_test['label']),

                 'sentence':df_test['sentence'].replace(r'\n',' ',regex=True)})

In [ ]:
# df_train_bert.to_csv('train.tsv', sep='\t', index=False, header=False)
# df_dev_bert.to_csv('dev.tsv', sep='\t', index=False, header=False)
df_test_bert.to_csv('test.tsv', sep='\t', index=False, header=True)

In [ ]:
df_test_bert.label.value_counts()

0     41076
16      474
1       240
28      204
96      202
      ...  
6         1
41        1
87        1
34        1
54        1
Name: label, Length: 97, dtype: int64

#Models

In [ ]:
!pip install --upgrade flair
!pip install sentence-transformers


     |████████████████████████████████| 322 kB 4.1 MB/s 
     |████████████████████████████████| 67 kB 5.0 MB/s 
     |████████████████████████████████| 3.8 MB 33.3 MB/s 
     |████████████████████████████████| 788 kB 40.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 19.7 MB 1.2 MB/s 
     |████████████████████████████████| 46 kB 3.9 MB/s 
     |████████████████████████████████| 48 kB 5.0 MB/s 
     |████████████████████████████████| 981 kB 37.2 MB/s 
     |████████████████████████████████| 53 kB 1.8 MB/s 
     |████████████████████████████████| 1.2 MB 44.5 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 596 kB 49.0 MB/s 
     |████████████████████████████████| 6.5 MB 38.1 MB/s 
     |████████████████████████████████| 895 kB 43.6 MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-py3-none-any.whl size=96

In [ ]:
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, FlairEmbeddings, BertEmbeddings, CharacterEmbeddings, DocumentRNNEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from typing import List
import pandas as pd
import os
import random
import xml.etree.cElementTree as ET
from sklearn.preprocessing import LabelEncoder
import nltk.data
from nltk import sent_tokenize
from flair.data_fetcher import NLPTaskDataFetcher
from pathlib import Path
from flair.data import Corpus
from flair.datasets import ClassificationCorpus
from flair.embeddings import SentenceTransformerDocumentEmbeddings


In [ ]:
import torch
print(torch.version.cuda)
print(torch.backends.cudnn.version())
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

11.1
8005
True
Tesla K80


In [ ]:
data_folder = Path('/content/dataset/')

# load corpus containing training, test, and dev data
corpus= ClassificationCorpus(data_folder,
                                          test_file='test.txt',
                                          dev_file='dev.txt',
                                          train_file='train.txt',
                                        )

2022-04-05 13:04:38,671 Reading data from /content/dataset
2022-04-05 13:04:38,677 Train: /content/dataset/train.txt
2022-04-05 13:04:38,682 Dev: /content/dataset/dev.txt
2022-04-05 13:04:38,702 Test: /content/dataset/test.txt
2022-04-05 13:04:40,499 Initialized corpus /content/dataset (label type name is 'class')


In [ ]:
label_dict = corpus.make_label_dictionary(label_type='class')

2022-04-05 13:04:40,520 Computing label dictionary. Progress:


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 49181/49181 [00:47<00:00, 1026.74it/s]

2022-04-05 13:05:28,659 Corpus contains the labels: class (#49181)
2022-04-05 13:05:28,662 Created (for label 'class') Dictionary with 92 tags: <unk>, Other, smoker.never, smoker.unknown, cad.test.before_dct, medication.aspirin..after_dct, medication.insulin..before_dct, diabetes.mention.after_dct, hypertension.high_bp.during_dct, hypertension.mention.before_dct, smoker.current, hyperlipidemia.mention.after_dct, medication.ace_inhibitor..before_dct, medication.beta_blocker..before_dct, medication.beta_blocker..during_dct, medication.statin..during_dct, medication.metformin..during_dct, diabetes.a1c.before_dct, cad.mention.after_dct, medication.insulin..during_dct, hyperlipidemia.high_ldl.before_dct, medication.nitrate..during_dct, hypertension.mention.during_dct, hypertension.mention.after_dct, cad.event.before_dct, medication.calcium_channel_blocker..after_dct, medication.aspirin..before_dct, medication.sulfonylureas..after_dct, medication.ace_inhibitor..after_dct, medication.sulfonyl

In [ ]:
embedding_types = [

   SentenceTransformerDocumentEmbeddings('microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext'),
]

embeddings = StackedEmbeddings(embeddings=embedding_types)

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.05k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/microsoft_BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
import flair
flair.device

device(type='cuda', index=0)

In [ ]:
from flair.models import TextClassifier
classifier = TextClassifier(embeddings, label_dictionary=label_dict,label_type='class', multi_label=True)


In [ ]:
trainer = ModelTrainer(classifier, corpus)


In [ ]:
trainer.train('/content/model_output/',
              learning_rate=0.1,
              mini_batch_size=16,
              anneal_factor=0.5,
              patience=5,
              max_epochs=10)

2022-04-05 15:04:41,893 ----------------------------------------------------------------------------------------------------
2022-04-05 15:04:41,900 Model: "TextClassifier(
  (loss_function): BCEWithLogitsLoss()
  (document_embeddings): StackedEmbeddings(
    (list_embedding_0): SentenceTransformerDocumentEmbeddings(
      (model): SentenceTransformer(
        (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
        (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
      )
    )
  )
  (decoder): Linear(in_features=768, out_features=92, bias=True)
  (weights): None
  (weight_tensor) None
)"
2022-04-05 15:04:41,912 ----------------------------------------------------------------------------------------------------
2022-04-05 15:04:41,917 Corpus: "Corpus: 49181 train + 5786 dev + 35723 test senten

/usr/local/lib/python3.7/dist-packages/flair/trainers/trainer.py:65: UserWarning: There should be no best model saved at epoch 1 except there is a model from previous trainings in your training folder. All previous best models will be deleted.
  "There should be no best model saved at epoch 1 except there is a model from previous trainings"
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


2022-04-05 15:06:01,015 epoch 1 - iter 307/3074 - loss 0.00061777 - samples/sec: 62.90 - lr: 0.100000
2022-04-05 15:07:18,356 epoch 1 - iter 614/3074 - loss 0.00063456 - samples/sec: 63.79 - lr: 0.100000
2022-04-05 15:08:37,534 epoch 1 - iter 921/3074 - loss 0.00066797 - samples/sec: 62.30 - lr: 0.100000
2022-04-05 15:09:55,720 epoch 1 - iter 1228/3074 - loss 0.00065688 - samples/sec: 63.35 - lr: 0.100000
2022-04-05 15:11:13,630 epoch 1 - iter 1535/3074 - loss 0.00066522 - samples/sec: 63.29 - lr: 0.100000
2022-04-05 15:12:31,437 epoch 1 - iter 1842/3074 - loss 0.00067078 - samples/sec: 63.38 - lr: 0.100000
2022-04-05 15:13:49,547 epoch 1 - iter 2149/3074 - loss 0.00066196 - samples/sec: 63.36 - lr: 0.100000
2022-04-05 15:15:07,425 epoch 1 - iter 2456/3074 - loss 0.00065788 - samples/sec: 63.30 - lr: 0.100000
2022-04-05 15:16:25,076 epoch 1 - iter 2763/3074 - loss 0.00065450 - samples/sec: 63.51 - lr: 0.100000
2022-04-05 15:17:42,683 epoch 1 - iter 3070/3074 - loss 0.00065917 - samples

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


2022-04-05 15:20:50,233 epoch 2 - iter 307/3074 - loss 0.00066839 - samples/sec: 64.04 - lr: 0.100000
2022-04-05 15:22:07,920 epoch 2 - iter 614/3074 - loss 0.00066915 - samples/sec: 63.77 - lr: 0.100000
2022-04-05 15:23:24,889 epoch 2 - iter 921/3074 - loss 0.00068582 - samples/sec: 64.07 - lr: 0.100000
2022-04-05 15:24:42,563 epoch 2 - iter 1228/3074 - loss 0.00066644 - samples/sec: 63.47 - lr: 0.100000
2022-04-05 15:25:59,691 epoch 2 - iter 1535/3074 - loss 0.00066356 - samples/sec: 64.15 - lr: 0.100000
2022-04-05 15:27:17,077 epoch 2 - iter 1842/3074 - loss 0.00066027 - samples/sec: 63.72 - lr: 0.100000
2022-04-05 15:28:34,578 epoch 2 - iter 2149/3074 - loss 0.00065723 - samples/sec: 63.63 - lr: 0.100000
2022-04-05 15:29:51,807 epoch 2 - iter 2456/3074 - loss 0.00065399 - samples/sec: 64.07 - lr: 0.100000
2022-04-05 15:31:08,682 epoch 2 - iter 2763/3074 - loss 0.00065532 - samples/sec: 64.13 - lr: 0.100000
2022-04-05 15:32:25,519 epoch 2 - iter 3070/3074 - loss 0.00065327 - samples

...

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


2022-04-05 17:17:51,145 epoch 10 - iter 307/3074 - loss 0.00066785 - samples/sec: 65.43 - lr: 0.100000
2022-04-05 17:19:06,813 epoch 10 - iter 614/3074 - loss 0.00062857 - samples/sec: 65.17 - lr: 0.100000
2022-04-05 17:20:23,934 epoch 10 - iter 921/3074 - loss 0.00062739 - samples/sec: 64.25 - lr: 0.100000
2022-04-05 17:21:40,659 epoch 10 - iter 1228/3074 - loss 0.00061847 - samples/sec: 64.26 - lr: 0.100000
2022-04-05 17:22:57,697 epoch 10 - iter 1535/3074 - loss 0.00060826 - samples/sec: 64.00 - lr: 0.100000
2022-04-05 17:24:15,044 epoch 10 - iter 1842/3074 - loss 0.00061147 - samples/sec: 63.98 - lr: 0.100000
2022-04-05 17:25:31,884 epoch 10 - iter 2149/3074 - loss 0.00060796 - samples/sec: 64.17 - lr: 0.100000
2022-04-05 17:26:49,292 epoch 10 - iter 2456/3074 - loss 0.00060890 - samples/sec: 63.70 - lr: 0.100000
2022-04-05 17:28:06,615 epoch 10 - iter 2763/3074 - loss 0.00061206 - samples/sec: 64.01 - lr: 0.100000
2022-04-05 17:29:23,802 epoch 10 - iter 3070/3074 - loss 0.00061491

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


2022-04-05 17:42:36,788 0.9321	0.8842	0.9075	0.8842
2022-04-05 17:42:36,790 
Results:
- F-score (micro) 0.9075
- F-score (macro) 0.0102
- Accuracy 0.8842

By class:
                                                precision    recall  f1-score   support

                                         Other     0.9321    0.9883    0.9593     31962
               hypertension.mention.before_dct     0.0000    0.0000    0.0000       241
                                smoker.unknown     0.0000    0.0000    0.0000       192
                    diabetes.mention.after_dct     0.0000    0.0000    0.0000       183
                 medication.statin..during_dct     0.0000    0.0000    0.0000       150
                          cad.event.before_dct     0.0000    0.0000    0.0000       144
                   diabetes.mention.before_dct     0.0000    0.0000    0.0000       112
               hypertension.high_bp.during_dct     0.0000    0.0000    0.0000       105
              hyperlipidemia.mention.after

{'dev_loss_history': [tensor(0.0006, device='cuda:0'),
  tensor(0.0006, device='cuda:0'),
  tensor(0.0006, device='cuda:0'),
  tensor(0.0006, device='cuda:0'),
  tensor(0.0006, device='cuda:0'),
  tensor(0.0006, device='cuda:0'),
  tensor(0.0006, device='cuda:0'),
  tensor(0.0006, device='cuda:0'),
  tensor(0.0006, device='cuda:0'),
  tensor(0.0006, device='cuda:0')],
 'dev_score_history': [0.9083855039238162,
  0.9086125241185757,
  0.9094273127753304,
  0.9092032118591723,
  0.9096848238721639,
  0.9097498453107046,
  0.9104701420128782,
  0.9106638418079096,
  0.911013215859031,
  0.9113767153607791],
 'test_score': 0.9075159455266334,
 'train_loss_history': [0.0006596786238814058,
  0.0006531360232342854,
  0.0006471647802616376,
  0.0006416947734426393,
  0.0006364509034419162,
  0.000631611803495703,
  0.0006270798309552481,
  0.0006227264817755734,
  0.0006186371603349503,
  0.0006147258182344029]}

In [ ]:
 !cp /content